In [2]:
from pycocotools.coco import COCO
import cv2
import numpy as np
import json
import torch
import os
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def array_to_gpu(array):
    tensor = torch.from_numpy(array).float().to(device)
    return tensor


def getSupercategory(category_id):
    f = open(coco_annotation_file_path)
    cat_json_file = json.load(f)

    for category in cat_json_file['categories']:
        if category_id == category['id']:
            f = open('../data/supercategories_top5_large_classes.json')
            supcat_json_file = json.load(f)
            return supcat_json_file[category['supercategory']]
            
coco_annotation_file_path = '../data/annotations.json'
coco_annotation = COCO(annotation_file=coco_annotation_file_path)      
img_ids = coco_annotation.getImgIds()

for index in range(len(img_ids)):
    
    img_id = img_ids[index]
    img_info = coco_annotation.loadImgs(img_id)[0]
    mask_filename = img_info["file_name"].replace('images/','').replace('.jpg', '.png').replace('.JPG', '.png')
    path_to_mask = '../data/processed/masks_5_classes/' + mask_filename
    
    if os.path.exists(path_to_mask):
        print(index, end='\r')
        pass
    else:
        print(index, end='\r')
        # Get masks
        ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
        anns = coco_annotation.loadAnns(ann_ids)
        mask = np.zeros((img_info['height'],img_info['width']))
        mask = array_to_gpu(mask)
        
        
        # Use supercategories instead classes
        for ann in anns:
            mask_from_annotation = array_to_gpu(coco_annotation.annToMask(ann))
            supercategory_of_cat = getSupercategory(ann['category_id'])
            mask = torch.maximum(mask, mask_from_annotation * supercategory_of_cat)

        mask = torch.unsqueeze(mask, axis=0)
        mask = torch.unsqueeze(mask, axis=0)
        mask = F.interpolate(mask, size=(512,512), mode='nearest')
        
        mask = torch.squeeze(mask, 0)
        mask = torch.squeeze(mask, 0)
        mask = mask.cpu().numpy()
        
        if not np.any(mask):
            pass
        else:
            cv2.imwrite(path_to_mask, mask)

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
